# Data Cleaning & Augmentation
Simplifying chords down to 42: base note (A-G) + accidental + major/minor(dim).
Transpose each song to mitigate bias from the key of the song and to augment our data.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
### Simplifying chords down to set of 42 ###
notes = ["A", "B", "C", "D", "E", "F", "G"]
accs = ["b", "s", ""]
all_notes_list = [note + acc for note in notes for acc in accs]

def simplify_chord(chord: str) -> str:
    """
    Removes chord quality from a chord.
    """
    for note in all_notes_list:
        if not chord.startswith(note):
            continue

        suffix = chord.removeprefix(note)
        if suffix.startswith("min") or suffix.startswith("dim"):
            return note + "min"
        else:
            return note

    if chord == "sC":
        return "Cs"

    # print(chord)
    return ""

In [ ]:
### Transposing songs to all keys ###
keys_list = ["C", "Cs", "D", "Ds", "E", "F", "Fs", "G", "Gs", "A", "As", "B"]

def transpose_chord(chord: str, variation: int) -> str:
    # Identify base note and suffix robustly
    if len(chord) >= 2 and chord[1] == "s":
        base = chord[:2]
        suffix = chord[2:]
    else:
        base = chord[:1]
        suffix = chord[1:]
    if base in keys_list:
        idx = keys_list.index(base)
        new_base = keys_list[(idx + variation) % 12]
        return new_base + suffix
    return chord  # If not found, return as is

def augment_keys(df):
    augmented_rows = []
    for _, row in df.iterrows():
        for variation in range(12):
            new_row = row.copy()
            if variation == 0:
                new_row["original_key"] = True
            else:
                new_row["original_key"] = False
            new_row["added_semitones"] = variation
            new_row["chords"] = [transpose_chord(chord, variation) for chord in row["chords"]]
            augmented_rows.append(new_row)
    return pd.DataFrame(augmented_rows)

def standardize_chord_prefix(chord: str) -> str:
    # Map flat notes to their sharp equivalents
    flat_to_sharp = {
        "Bb": "As",
        "Db": "Cs",
        "Eb": "Ds",
        "Gb": "Fs",
        "Ab": "Gs",
        "Bs": "C",
        "Es": "F"
    }
    for flat, sharp in flat_to_sharp.items():
        if chord.startswith(flat):
            return sharp + chord[len(flat):]
    return chord

In [ ]:
# Writing transposed dataset to csv
def get_pop_chords_df():
    df = pd.read_csv("hf://datasets/ailsntua/Chordonomicon/chordonomicon_v2.csv",usecols=["id", "chords", "main_genre"])
    pop_df = df[df["main_genre"] == "pop"][["id", "chords"]].copy()
    pop_df["chords"] = pop_df["chords"].str.split(" ")
    pop_df["chords"] = pop_df["chords"].map(
        lambda chords: [simplify_chord(standardize_chord_prefix(chord)) for chord in chords if not chord.startswith("<")]
    )

    pop_df["original_key"] = True
    pop_df["added_semitones"] = 0
    return pop_df

pop_chords_df = get_pop_chords_df()
augmented_df = augment_keys(pop_chords_df)
print(augmented_df.head(15))
augmented_df.to_csv("chordonomicon_v2_augmented.csv", index=False)

# N-gram Learning
Compute n-gram counts using CountVectorizer library (usually used for bag of n-grams)

In [ ]:
### Calculating n-gram counts ###
# def load_chord_data(series) -> tuple[dict[str, int], dict[str, dict[str, int]]]:
#     unigram_dict: dict[str, int] = {}
#     bigram_dict: dict[str, dict[str, int]] = {}

#     for row in series:
#         # Unigram counts
#         for chord in row:
#             unigram_dict[chord] = unigram_dict.get(chord, 0) + 1
#         # Bigram counts
#         for i in range(len(row) - 1):
#             w1, w2 = row[i], row[i + 1]
#             if w1 not in bigram_dict:
#                 bigram_dict[w1] = {}
#             bigram_dict[w1][w2] = bigram_dict[w1].get(w2, 0) + 1

#     return unigram_dict, bigram_dict

def count_n_grams(data, n: int = 1) -> pd.DataFrame:
    word_vectorizer = CountVectorizer(
        ngram_range=(1, n),
        analyzer="word",
        token_pattern=r"(?u)\b\w+\b",
        lowercase=False,
    )

    sparse_matrix = word_vectorizer.fit_transform(
        data.map(lambda chords: " ".join(chords))
    )

    frequencies = sum(sparse_matrix).toarray()[0]

    df_all = pd.DataFrame(
        frequencies,
        index=word_vectorizer.get_feature_names_out(),
        columns=["count"],
    )

    return df_all.groupby(by=lambda chords: len(chords.split(" ")))

Import our processed dataset and compute n-gram counts.

In [53]:
import ast

# transposed pop songs
pop_chords = pd.read_csv('chordonomicon_v2_augmented.csv', usecols=['chords'])['chords'].apply(ast.literal_eval)

# n hyperparameter
n = 3

# counts for all n-grams up to n
n_gram_counts = count_n_grams(pop_chords, n)
for key, _ in n_gram_counts:
    print(n_gram_counts.get_group(key).sort_values(by='count'), "\n\n")

         count
Dmin   1999731
Gmin   1999731
Fsmin  1999731
Fmin   1999731
Emin   1999731
Dsmin  1999731
Csmin  1999731
Gsmin  1999731
Bmin   1999731
Asmin  1999731
Amin   1999731
Cmin   1999731
Cs     4767427
C      4767427
D      4767427
Gs     4767427
Ds     4767427
E      4767427
B      4767427
F      4767427
Fs     4767427
As     4767427
G      4767427
A      4767427 


              count
Dmin Gsmin     1419
Fmin Bmin      1419
Bmin Fmin      1419
Emin Asmin     1419
Asmin Emin     1419
...             ...
F C         1034487
Ds As       1034487
D A         1034487
E B         1034487
G D         1034487

[576 rows x 1 columns] 


                 count
Csmin Amin Cs        4
Cmin Gsmin C         4
Emin Cmin E          4
Dmin Asmin D         4
Asmin Fsmin As       4
...                ...
Ds Gs Ds        264009
Cs Fs Cs        264009
F As F          264009
D G D           264009
A D A           264009

[13812 rows x 1 columns] 




In [59]:
import itertools

# all possible chords (flats consolidated with sharps)
notes = ["A", "B", "C", "D", "E", "F", "G"]
accs = ["", "s"]
third = ["", "min"]
excluded_chords = ['Bs', 'Bsmin', 'Es', 'Esmin']
all_chords_list = [note + acc + t for note in notes for acc in accs for t in third]
all_chords_list = [c for c in all_chords_list if c not in excluded_chords]
print(all_chords_list)
print(len(all_chords_list))

['A', 'Amin', 'As', 'Asmin', 'B', 'Bmin', 'C', 'Cmin', 'Cs', 'Csmin', 'D', 'Dmin', 'Ds', 'Dsmin', 'E', 'Emin', 'F', 'Fmin', 'Fs', 'Fsmin', 'G', 'Gmin', 'Gs', 'Gsmin']
24


In [66]:
# See observed n-grams for each n
for i in range(1,n+1):
    observed_ngrams = n_gram_counts.get_group(i).index.unique().tolist()
    print(f"Observed {len(observed_ngrams)} {i}-grams: " + str(observed_ngrams))

    # not observed
    all_possible_ngrams = {" ".join(c) for c in itertools.product(all_chords_list, repeat=i)}
    observed_ngrams = set(observed_ngrams)
    missing = all_possible_ngrams - observed_ngrams
    if missing:
        print(f"Missing {i}-grams: " + str(missing))

Observed 24 1-grams: ['A', 'Amin', 'As', 'Asmin', 'B', 'Bmin', 'C', 'Cmin', 'Cs', 'Csmin', 'D', 'Dmin', 'Ds', 'Dsmin', 'E', 'Emin', 'F', 'Fmin', 'Fs', 'Fsmin', 'G', 'Gmin', 'Gs', 'Gsmin']
Observed 576 2-grams: ['A A', 'A Amin', 'A As', 'A Asmin', 'A B', 'A Bmin', 'A C', 'A Cmin', 'A Cs', 'A Csmin', 'A D', 'A Dmin', 'A Ds', 'A Dsmin', 'A E', 'A Emin', 'A F', 'A Fmin', 'A Fs', 'A Fsmin', 'A G', 'A Gmin', 'A Gs', 'A Gsmin', 'Amin A', 'Amin Amin', 'Amin As', 'Amin Asmin', 'Amin B', 'Amin Bmin', 'Amin C', 'Amin Cmin', 'Amin Cs', 'Amin Csmin', 'Amin D', 'Amin Dmin', 'Amin Ds', 'Amin Dsmin', 'Amin E', 'Amin Emin', 'Amin F', 'Amin Fmin', 'Amin Fs', 'Amin Fsmin', 'Amin G', 'Amin Gmin', 'Amin Gs', 'Amin Gsmin', 'As A', 'As Amin', 'As As', 'As Asmin', 'As B', 'As Bmin', 'As C', 'As Cmin', 'As Cs', 'As Csmin', 'As D', 'As Dmin', 'As Ds', 'As Dsmin', 'As E', 'As Emin', 'As F', 'As Fmin', 'As Fs', 'As Fsmin', 'As G', 'As Gmin', 'As Gs', 'As Gsmin', 'Asmin A', 'Asmin Amin', 'Asmin As', 'Asmin Asmin',

Calculate transition matrix probabilities using counts.

In [81]:
# Calculate transition matrix probabilities
# alpha is additive smoothing

def compute_unigram_prob(n_gram_counts, alpha=1.0):
    unigram = n_gram_counts.get_group(1)
    unigram = unigram.reindex(all_chords_list, fill_value=0)
    vocab_size = len(all_chords_list)
    total_count = unigram["count"].sum()

    probs = (unigram["count"] + alpha) / (total_count + alpha * vocab_size)
    df = pd.DataFrame([probs.values], 
                      index=[""],
                      columns=all_chords_list)
    return df

# def compute_bigram_prob(n_gram_counts, alpha=1.0):
#     bigram = n_gram_counts.get_group(2)
#     bigram["evidence"] = bigram.index.map(lambda s: s.split()[0]) # get (n-1)-length evidence
#     bigram["next"] = bigram.index.map(lambda s: s.split()[1]) # next chords

#     full_index = pd.MultiIndex.from_product([all_chords_list, all_chords_list], names=["evidence", "next"])
#     bigram = bigram.set_index(["evidence", "next"])
#     bigram = bigram.reindex(full_index, fill_value=0)

#     evidence_counts = bigram["count"].groupby(level="evidence").transform("sum")
#     num_next = bigram.index.get_level_values("next").nunique()
#     bigram["prob"] = (bigram["count"] + alpha) / (evidence_counts + alpha * num_next)

#     # 2d dataframe
#     return bigram["prob"].unstack(fill_value=0.0)

def compute_ngram_prob(n_gram_counts, n : int = 2, alpha=1.0):
    ngram = n_gram_counts.get_group(n)

    ngram["evidence"] = ngram.index.map(lambda s: " ".join(s.split()[:-1]))
    ngram["next"] = ngram.index.map(lambda s: s.split()[-1])
    
    # generate all possible (n-1)-length chord sequeneces
    all_evidence_seq = [" ".join(evidence) for evidence in itertools.product(all_chords_list, repeat=(n - 1))]
    full_index = pd.MultiIndex.from_product([all_evidence_seq, all_chords_list], names=["evidence", "next"])

    # reindex to (24^(n-1), n)
    ngram = ngram.set_index(["evidence", "next"])
    ngram = ngram.reindex(full_index, fill_value=0)

    # compute probs
    evidence_counts = ngram["count"].groupby(level="evidence").transform("sum")
    vocab_size = len(all_chords_list)
    ngram["prob"] = (ngram["count"] + alpha) / (evidence_counts + alpha * vocab_size)

    # 2d df transition matrix
    return ngram["prob"].unstack(fill_value=0.0)

In [80]:
unigram_probs = compute_unigram_prob(n_gram_counts)
print(unigram_probs.shape)
unigram_probs

(1, 24)


,A,Amin,As,Asmin,B,Bmin,C,Cmin,Cs,Csmin,...,E,Emin,F,Fmin,Fs,Fsmin,G,Gmin,Gs,Gsmin
,0.058708,0.024625,0.058708,0.024625,0.058708,0.024625,0.058708,0.024625,0.058708,0.024625,...,0.058708,0.024625,0.058708,0.024625,0.058708,0.024625,0.058708,0.024625,0.058708,0.024625


In [69]:
bigram_probs = compute_ngram_prob(n_gram_counts, 2)
print(bigram_probs.shape)
bigram_probs

(24, 24)


/var/folders/c8/zh68_k6j30l283wd6sjg39280000gn/T/ipykernel_25722/4147788041.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ngram["evidence"] = ngram.index.map(lambda s: " ".join(s.split()[:-1]))
/var/folders/c8/zh68_k6j30l283wd6sjg39280000gn/T/ipykernel_25722/4147788041.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ngram["next"] = ngram.index.map(lambda s: s.split()[-1])


next,A,Amin,As,Asmin,B,Bmin,C,Cmin,Cs,Csmin,...,E,Emin,F,Fmin,Fs,Fsmin,G,Gmin,Gs,Gsmin
evidence,,,,,,,,,,,,,,,,,,,,,
A,0.024163,0.006444,0.013058,0.002419,0.096524,0.091456,0.013914,0.001180,0.012781,0.056525,...,0.220030,0.025416,0.008502,0.000809,0.012630,0.074177,0.079297,0.003074,0.018900,0.012503
Amin,0.007904,0.015830,0.055139,0.002382,0.021342,0.021545,0.137630,0.003397,0.001412,0.001666,...,0.032427,0.087222,0.194492,0.002060,0.002720,0.002243,0.179469,0.021817,0.006409,0.003352
As,0.018900,0.012503,0.024163,0.006444,0.013058,0.002419,0.096524,0.091456,0.013914,0.001180,...,0.004494,0.001372,0.220030,0.025416,0.008502,0.000809,0.012630,0.074177,0.079297,0.003074
Asmin,0.006409,0.003352,0.007904,0.015830,0.055139,0.002382,0.021342,0.021545,0.137630,0.003397,...,0.004538,0.000717,0.032427,0.087222,0.194492,0.002060,0.002720,0.002243,0.179469,0.021817
B,0.079297,0.003074,0.018900,0.012503,0.024163,0.006444,0.013058,0.002419,0.096524,0.091456,...,0.188363,0.031969,0.004494,0.001372,0.220030,0.025416,0.008502,0.000809,0.012630,0.074177
Bmin,0.179469,0.021817,0.006409,0.003352,0.007904,0.015830,0.055139,0.002382,0.021342,0.021545,...,0.090660,0.103629,0.004538,0.000717,0.032427,0.087222,0.194492,0.002060,0.002720,0.002243
C,0.012630,0.074177,0.079297,0.003074,0.018900,0.012503,0.024163,0.006444,0.013058,0.002419,...,0.012781,0.056525,0.188363,0.031969,0.004494,0.001372,0.220030,0.025416,0.008502,0.000809
Cmin,0.002720,0.002243,0.179469,0.021817,0.006409,0.003352,0.007904,0.015830,0.055139,0.002382,...,0.001412,0.001666,0.090660,0.103629,0.004538,0.000717,0.032427,0.087222,0.194492,0.002060
Cs,0.008502,0.000809,0.012630,0.074177,0.079297,0.003074,0.018900,0.012503,0.024163,0.006444,...,0.013914,0.001180,0.012781,0.056525,0.188363,0.031969,0.004494,0.001372,0.220030,0.025416


In [77]:
trigram_probs = compute_ngram_prob(n_gram_counts, 3) # trigram
print(trigram_probs.shape)
trigram_probs

(576, 24)


/var/folders/c8/zh68_k6j30l283wd6sjg39280000gn/T/ipykernel_25722/4147788041.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ngram["evidence"] = ngram.index.map(lambda s: " ".join(s.split()[:-1]))
/var/folders/c8/zh68_k6j30l283wd6sjg39280000gn/T/ipykernel_25722/4147788041.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ngram["next"] = ngram.index.map(lambda s: s.split()[-1])


next,A,Amin,As,Asmin,B,Bmin,C,Cmin,Cs,Csmin,...,E,Emin,F,Fmin,Fs,Fsmin,G,Gmin,Gs,Gsmin
evidence,,,,,,,,,,,,,,,,,,,,,
A A,0.262934,0.004893,0.012642,0.001361,0.032749,0.041379,0.008514,0.000863,0.006317,0.023986,...,0.072321,0.011530,0.009920,0.000641,0.010124,0.102517,0.031655,0.003025,0.014395,0.007660
A Amin,0.080751,0.015212,0.010655,0.001340,0.050930,0.024694,0.041615,0.004557,0.004054,0.031262,...,0.374033,0.052136,0.031127,0.000938,0.004456,0.018462,0.071570,0.004624,0.007773,0.036120
A As,0.302351,0.012841,0.020520,0.018514,0.038014,0.020848,0.144541,0.012792,0.008698,0.004702,...,0.028214,0.006116,0.128543,0.002302,0.006001,0.014913,0.036501,0.048175,0.011986,0.001381
A Asmin,0.132618,0.003016,0.009137,0.021024,0.044176,0.126053,0.011887,0.011088,0.074514,0.008693,...,0.052515,0.003637,0.023241,0.021467,0.226736,0.008693,0.058991,0.002218,0.060853,0.009847
A B,0.147487,0.000795,0.001590,0.000732,0.013254,0.004759,0.010107,0.001845,0.023267,0.241010,...,0.374321,0.008857,0.002311,0.000246,0.022608,0.038910,0.013624,0.000346,0.014015,0.046487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gsmin Fsmin,0.117987,0.009444,0.001049,0.002075,0.251644,0.011519,0.006599,0.000816,0.009793,0.123280,...,0.189386,0.005293,0.003031,0.002938,0.009071,0.012428,0.001632,0.000793,0.026839,0.187893
Gsmin G,0.043899,0.004922,0.008891,0.002620,0.170914,0.024609,0.022228,0.028419,0.025482,0.028896,...,0.154561,0.017544,0.046122,0.003096,0.176629,0.061761,0.013495,0.004684,0.006589,0.081607
Gsmin Gmin,0.012852,0.020865,0.030692,0.012247,0.020109,0.006501,0.179770,0.119444,0.012247,0.012549,...,0.019050,0.001966,0.048533,0.050801,0.049894,0.232386,0.010432,0.019958,0.019958,0.059873


# Inference
Deterministic and probabilistic methods.

In [90]:
def deterministic_inference(evidence):
    # evidence: string of n-1 space-separated chords
    
    n = len(evidence.split()) + 1
    ngram_probs = unigram_probs if n == 1 else (bigram_probs if n == 2 else trigram_probs)

    if evidence not in ngram_probs.index:
        raise KeyError(f"Evidence '{evidence}' not found in {n}-gram table")
    
    row_probs = ngram_probs.loc[evidence]
    return row_probs.idxmax() # returns next chord w highest prob, if there are several, the first one in col order

def probabilistic_inference(evidence):
    # evidence: string of n-1 space-separated chords
    
    n = len(evidence.split()) + 1
    ngram_probs = unigram_probs if n == 1 else (bigram_probs if n == 2 else trigram_probs)

    if evidence not in ngram_probs.index:
        raise KeyError(f"Evidence '{evidence}' not found in {n}-gram table")
    
    row_probs = ngram_probs.loc[evidence]
    cdf = np.cumsum(row_probs.values) # create cumulative distribution over next possible chord

    # sample over dist
    seed = np.random.random()
    idx = np.searchsorted(cdf, seed)
    
    return row_probs.index[idx] # return probabilistically chosen next chord

In [ ]:
### test inference for bigram ###
seq = []

for _ in range(16):
    if len(seq) == 0:
        evidence = ""
    elif len(seq) == 1:
        evidence = seq[-1]
    else:
        evidence = " ".join(seq[-2:])

    next_chord = probabilistic_inference(evidence) # can change to deterministic_inference()
    seq.append(next_chord)

print(seq)

['Csmin', 'Gs', 'A', 'E', 'Bmin', 'D', 'A', 'Fsmin', 'Csmin', 'E', 'A', 'D', 'Fsmin', 'D', 'A', 'Fsmin']


# Evaluation
Evaluate log-likelihood of an n-gram given a song

In [105]:
def song_log_likelihood_ngram(song, n, ngram_probs):
    # song: list of chords in song
    # n: order of the n-gram model
    # ngram_probs: dict[context_tuple] -> dict[target] = P(target | context)
    # ex: trigram ngram_prob = dict[(chord1, chord2)] = {chord0:P,...,chordV:P}, dict[chord3] = P(chord3 | chord1, chord2)
    # vocab_size: 42 or 36?

    ll = 0.0
    if len(song) < n:
        return 0.0

    for t in range(n-1, len(song)):
        if n == 1:
            context = ""
        else:
            context = " ".join(song[t-(n-1):t])

        target = song[t]

        try:
            p = ngram_probs.loc[context, target]
        except KeyError:
            print(f"KeyError given evidence {context}")
            p = 1e-12

        if p <= 0:
            p = 1e-12
        
        ll += np.log(p)

    return ll

In [106]:
# test log-likelihood
print(song_log_likelihood_ngram(seq, 2, bigram_probs))

-35.438954926606584


In [ ]:
def top_k_accuracy_ngram(song, n, ngram_probs, k=5):
    # song: list of chords in song
    # n: order of the n-gram model
    # ngram_probs: DataFrame with index=evidence, columns=next chords
    # k: number of top predictions to consider
    
    correct = 0
    total = 0
    
    if len(song) < n:
        return 0.0
    
    for t in range(n-1, len(song)):
        if n == 1:
            context = ""
        else:
            context = " ".join(song[t-(n-1):t])
        
        target = song[t]
        
        try:
            prob_row = ngram_probs.loc[context]
            
            top_k_chords = prob_row.nlargest(k).index.tolist()
            
            if target in top_k_chords:
                correct += 1
            total += 1
            
        except KeyError:
            total += 1
    
    return correct / total if total > 0 else 0.0

In [37]:
# Test on top-k accuracy
print(f"Top-1: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=1):.4f}")
print(f"Top-3: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=3):.4f}")
print(f"Top-5: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=5):.4f}")

Top-1: 0.1333
Top-3: 0.4000
Top-5: 0.6000
